<a href="https://colab.research.google.com/github/UoA-eResearch/VisSuiteQR/blob/main/ExcelQRLabels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VisSuite Label generation (QR Code)

This notebook is intended to facilitate the creation of printed labels to be stuck on eqipment.

The latest version of a spreadsheet needs to be uploaded (see [README](https://github.com/UoA-eResearch/VisSuiteQR/blob/main/README.md))

| Size        | Usage                           |
|-------------|---------------------------------|
| 'Big'     | ... on boxes                    |
| 'Small'   | ... on headsets, etc            |
| 'Super Small' | ... on controllers          |

![exampleLabels](https://raw.githubusercontent.com/UoA-eResearch/VisSuiteQR/main/Labels.jpg)

## Process
This description is based on the understanding that you have the printer (Brother Q700), etc. successfully installed on your local computer and connected it.

- open this Colab Notebook
- drag and drop the (Sharepoint downloaded) `xslx` file into the files section
- ![dragdrop](https://raw.githubusercontent.com/UoA-eResearch/VisSuiteQR/main/DragDrop.jpg)
- as this spreadsheet is continued (new kit gets added to later rows), you probably want to specify the range (*start from row x until row y*), for this, use the boxes below in Section "Pick what you need"
- run the whole notebook `Runtime`, `Run all`
- download the generated PDF from the Files section (leftmost sidebar, folder icon)
- open these in Adobe Acrobat
- Choose `File` then `Print`
  - make sure to select the correct printer, then select `Page setup` and dial in the settings as shown in the following screenshots:

### Settings for Big Labels

![big](https://raw.githubusercontent.com/UoA-eResearch/VisSuiteQR/main/AcrobatBig.jpg)

### Settings for Small Labels

![small](https://raw.githubusercontent.com/UoA-eResearch/VisSuiteQR/main/AcrobatSmall.jpg)


### Settings for Super Small Labels

![supersmall](https://raw.githubusercontent.com/UoA-eResearch/VisSuiteQR/main/AcrobatSuperSmall.jpg)





## Import needed libraries

In [ ]:
import openpyxl
import pandas as pd
import requests
import shutil

# Pick what you need

- To avoid repetition ( [DRY principle](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)), data (such as names, serial numbers and short names are only to be entered once into the spreadsheet
- as this sheet keeps growing but you most likely only want to print labels for the most recently added items, you can specify the row numbers (as per the Excel sheet, starting with 1) that you want to print labels for

In [1]:
#@markdown ## Enter your desired colums here
start_row = "103" #@param {type: "string"}
end_row = "108" #@param {type: "string"}

todo adapt to Sharepoint ingest Excel

## Load the Excel sheet as a dataframe

In [ ]:
# From VisLog.xlsx and sheet Inventory
workbook = openpyxl.load_workbook('VisLog.xlsx')
sheet = workbook['Inventory']

# Convert input strings to integers
start_row = int(start_row) if start_row else None
end_row = int(end_row) if end_row else None

data = []
if start_row and end_row:
  # Get column names from row 1
  column_names = [cell.value for cell in sheet[1]]

  for row in sheet.iter_rows(min_row=start_row, max_row=end_row, values_only=True):
    data.append(row)

  df = pd.DataFrame(data, columns=column_names)  # Use column names from row 1



# Create Big Labels for the Boxes, etc

## Translate the previously imported information into ZPL

In [ ]:
big_label_list = []

for _, row in df.iterrows():
    big_label_list.append(
        f"^XA^XFR:SAMPLE.ZPL^FN1^FD{row['Item']}^FS^FN2^FD{row['UoA Tag']}^FS"
        f" ^FN3^FD{row['Short Number']}^FS ^FN4^FD{row['Short Name']}^FS"
        f" ^FN5^FD{row['Serial Number']}^FS ^FN6^FDQA{row['UoA Tag']} ^XZ"
    )

print(big_label_list)

['^XA^XFR:SAMPLE.ZPL^FN1^FDHoloLensV1^FS^FN2^FDUOA316072^FS ^FN3^FD#3^FS ^FN4^FDHoloV1^FS ^FN5^FD991263056^FS ^FN6^FDQAUOA316072 ^XZ', '^XA^XFR:SAMPLE.ZPL^FN1^FDHoloLensV1^FS^FN2^FDUOA316093^FS ^FN3^FD#4^FS ^FN4^FDHoloV1^FS ^FN5^FD660264756^FS ^FN6^FDQAUOA316093 ^XZ', '^XA^XFR:SAMPLE.ZPL^FN1^FDHoloLensV1^FS^FN2^FDUOA316095^FS ^FN3^FD#5^FS ^FN4^FDHoloV1^FS ^FN5^FD613464956^FS ^FN6^FDQAUOA316095 ^XZ', '^XA^XFR:SAMPLE.ZPL^FN1^FDHoloLensV1^FS^FN2^FDUOA316197^FS ^FN3^FD#6^FS ^FN4^FDHoloV1^FS ^FN5^FD503665056^FS ^FN6^FDQAUOA316197 ^XZ', '^XA^XFR:SAMPLE.ZPL^FN1^FDVR LAPTOP NEW^FS^FN2^FDUOA316123^FS ^FN3^FD^FS ^FN4^FDNone^FS ^FN5^FDBY2210M78901043^FS ^FN6^FDQAUOA316123 ^XZ', '^XA^XFR:SAMPLE.ZPL^FN1^FDElgato Stream Deck Mini^FS^FN2^FDUOA448133^FS ^FN3^FD^FS ^FN4^FDNone^FS ^FN5^FDBL22L2B05133^FS ^FN6^FDQAUOA448133 ^XZ']


## Use the previously generated information in a templated layout (the box with the characteristic UoA angled corners)

In [ ]:
# ZPL Labels:
zpltemplate = '^XA (600 dots wide; 372 dots tall, 7dots border 88dots angle) ^DFR:SAMPLE.ZPL^FS (this is the template bit) ^FO 80,35^A0,,35^FD CeR - Centre for eResearch ^FS (this is the title) ^FO7,7^GD88,88,10,B,^FS (this is the top-left diagonal) ^FO488,275^GD88,88,10,B,^FS (this is the bottom-right diagonal) ^FO7,95^GB10,269,10^FS (this is the left line) ^FO576,7^GB10,269,10^FS (this is the right line) ^FO95,7^GB488,10,10,^FS (this is the top line) ^FO7,355^GB488,10,10,^FS (this is the bottom line) ^FO350,65^BQN,2,9 ^FN6 ^FS  (this is the QR code) ^FO 80,125^A0,,18^FDItem: ^FS (Static Text Item:) ^FO190,125^ADN,18,10^FN1^FS (ItemInsert) ^FO 80,165^A0,,18^FDUoA Tag No.^FS (Static TextUoA Tag No.) ^FO190,165^ADN,18,10^FN2^FS (UoATagInsert) ^FO 80,205^A0,,18^FDShort:^FS (Static Text Short:) ^FO190,205^A0,36,30^FN4^FS (ShortInsert) ^FO190,245^A0,72,60^FN3^FS (ShortInsertNUMBER) ^FO 80,320^A0,,12^FDSerial:^FS (Static Text Short:) ^FO190,320^A0,,25^FN5^FS SerialInsert) ^XZ'

# Prepend the template string to the list
big_label_list.insert(0, zpltemplate)

# Flatten the list into a single string
big_label_all = ''.join(big_label_list)

# Output the final string
print(big_label_all)



^XA (600 dots wide; 372 dots tall, 7dots border 88dots angle) ^DFR:SAMPLE.ZPL^FS (this is the template bit) ^FO 80,35^A0,,35^FD CeR - Centre for eResearch ^FS (this is the title) ^FO7,7^GD88,88,10,B,^FS (this is the top-left diagonal) ^FO488,275^GD88,88,10,B,^FS (this is the bottom-right diagonal) ^FO7,95^GB10,269,10^FS (this is the left line) ^FO576,7^GB10,269,10^FS (this is the right line) ^FO95,7^GB488,10,10,^FS (this is the top line) ^FO7,355^GB488,10,10,^FS (this is the bottom line) ^FO350,65^BQN,2,9 ^FN6 ^FS  (this is the QR code) ^FO 80,125^A0,,18^FDItem: ^FS (Static Text Item:) ^FO190,125^ADN,18,10^FN1^FS (ItemInsert) ^FO 80,165^A0,,18^FDUoA Tag No.^FS (Static TextUoA Tag No.) ^FO190,165^ADN,18,10^FN2^FS (UoATagInsert) ^FO 80,205^A0,,18^FDShort:^FS (Static Text Short:) ^FO190,205^A0,36,30^FN4^FS (ShortInsert) ^FO190,245^A0,72,60^FN3^FS (ShortInsertNUMBER) ^FO 80,320^A0,,12^FDSerial:^FS (Static Text Short:) ^FO190,320^A0,,25^FN5^FS SerialInsert) ^XZ^XA^XFR:SAMPLE.ZPL^FN1^FDHoloL

In [ ]:
url = 'http://api.labelary.com/v1/printers/12dpmm/labels/1.94x1.22/'
files = {'file' : big_label_all}
headers = {'Accept' : 'application/pdf'} # omit this line to get PNG images back
response = requests.post(url, headers = headers, files = files, stream = True)

if response.status_code == 200:
    response.raw.decode_content = True
    with open('QRlabelsBig.pdf', 'wb') as out_file: # change file name for PNG images
        shutil.copyfileobj(response.raw, out_file)
else:
    print('Error: ' + response.text)

# Create small labels for Headsets, etc.

In [ ]:
small_label_list = []
for index, row in df.iterrows():
    small_label_list.append(f"^XA^XFR:SAMPLE.ZPL^FN3^FD{row['Short Number']}^FS^FN4^FD{row['Short Name']}^FS ^FN6^FD{row['UoA Tag']}^FS ^XZ")

# print(short_label_list)

In [ ]:
zpl_small_template = '^XA (600 dots wide; 372 dots tall, 7dots border 88dots angle) ^DFR:SAMPLE.ZPL^FS (this is the template bit) ^FO 80,35^A0,,35^FD CeR - Centre for eResearch ^FS (this is the title) ^FO7,7^GD88,88,10,B,^FS (this is the top-left diagonal) ^FO488,185^GD88,88,10,B,^FS (this is the bottom-right diagonal) ^FO7,95^GB10,180,10^FS (this is the left line) ^FO576,7^GB10,180,10^FS (this is the right line) ^FO95,7^GB488,10,10,^FS (this is the top line) ^FO7,265^GB488,10,10,^FS (this is the bottom line) ^FO300,65^BQN,2,8 ^FN6 ^FS  (this is the QR code) ^FO80,80^A0,54,45^FN4^FS (ShortInsert) ^FO80,145^A0,120,100^FN3^FS (ShortInsertNUMBER) ^XZ'
# zplvariable = 'HoloLensV2^FS^FN2^FDUOA123123^FS (UoATagInsert) ^FN3^FDHoloV2^FS(ShortInsert) ^FN4^FD#1^FS(ShortInsertNUMBER) ^FN5^FDSERIAL123123123123^FS(SerialNo) ^XZ'

zpl_small = zpl_small_template + str(small_label_list)
# Brother QL 700 can do 300dpi, that is about 11.8dpmm, nearest of set of posibilities (Valid values are 6dpmm, 8dpmm, 12dpmm, and 24dpmm) is 12
# adjust print density (12dpmm), label width (3.94 inches), label height (2.44 inches), and label index (0) as necessary
# 1200 dots wide; 744 dots tall
# but: QR Codes can only be scaled with internal factors 1-10, that's to small, so half everything
url = 'http://api.labelary.com/v1/printers/12dpmm/labels/1.96x0.94/'
# files = {'file' : (zpl, zplb)}
files = {'file' : zpl_small}
headers = {'Accept' : 'application/pdf'} # omit this line to get PNG images back
response = requests.post(url, headers = headers, files = files, stream = True)

if response.status_code == 200:
    response.raw.decode_content = True
    with open('QRlabelsSmall.pdf', 'wb') as out_file: # change file name for PNG images
        shutil.copyfileobj(response.raw, out_file)
else:
    print('Error: ' + response.text)

# Create Controller labels (really small)

In [ ]:
super_small_template = "^XA ^DFR:SAMPLEsuper.ZPL^FS (this is the template bit)^CF0,50,50 ^PR12 ^LRY ^MD30 ^PW495 ^LL228 ^PON ^FO17,17^GD38,38,10,B,^FS (this is the top-left diagonal) ^FO250,17^GD38,38,10,B,^FS (this is the top-left diagonal) ^FO19,55^GB8,150,6^FS (this is the left line) ^FO250,55^GB8,150,6^FS (this is the left line) ^FO240,17^GB8,190,6^FS (this is the right line) ^FO470,17^GB8,190,6^FS (this is the right line) ^FO58,17^GB186,10,10,^FS (this is the top line) ^FO288,17^GB186,10,10,^FS (this is the top line) ^FO19,200^GB226,10,10,^FS (this is the bottom line) ^FO248,200^GB226,10,10,^FS (this is the bottom line) ^FO51,40^FN1^FS ^FO51,90^FN2^FS ^FO51,131^FDleft^FS ^FO282,40^FN3^FS ^FO282,90^FN4^FS ^FO282,131 ^FDright^FS ^XZ"

In [ ]:
super_small_label_list = []
for index, row in df.iterrows():
    super_small_label_list.append(f"^XA^XFR:SAMPLEsuper.ZPL^FN1^FD{row['Short Name']}^FS^FN2^FD{row['Short Number']}^FS^FN3^FD{row['Short Name']}^FS^FN4^FD{row['Short Number']}^FS ^XZ")

# # print(super_small_label_list)


# # Prepend the template string to the list
# super_small_label_list.insert(0, super_small_template)

# # Flatten the list into a single string
# super_small_label_all = ''.join(super_small_label_list)

# # Output the final string
# print(super_small_label_all)

In [ ]:
zpl_super_small = super_small_template + str(super_small_label_list)
# Brother QL 700 can do 300dpi, that is about 11.8dpmm, nearest of set of posibilities (Valid values are 6dpmm, 8dpmm, 12dpmm, and 24dpmm) is 12
# adjust print density (12dpmm), label width (3.94 inches), label height (2.44 inches), and label index (0) as necessary
# 1200 dots wide; 744 dots tall
# but: QR Codes can only be scaled with internal factors 1-10, that's to small, so half everything
url = 'http://api.labelary.com/v1/printers/8dpmm/labels/2.44x1.14/'
# files = {'file' : (zpl, zplb)}
files = {'file' : zpl_super_small}
headers = {'Accept' : 'application/pdf'} # omit this line to get PNG images back
response = requests.post(url, headers = headers, files = files, stream = True)

if response.status_code == 200:
    response.raw.decode_content = True
    with open('QRlabelsSuperSmall.pdf', 'wb') as out_file: # change file name for PNG images
        shutil.copyfileobj(response.raw, out_file)
else:
    print('Error: ' + response.text)

In [ ]:
# alternative approach instead of API: Local PDF genreation; challenge: zplgrf currently can only output pixel
# !pip install zplgrf